# Experiment 5: Two-Step Learning

In [ ]:
from os import chdir, getcwd

if not getcwd().lower().endswith("gb-birp"):
    chdir("..")

In [ ]:
RUN_ID = 1
BATCH_SIZE = 64
EPOCHS = 50
INITIAL_LR = 1e-3
END_LR = 1e-5

In [ ]:
import math
import pickle
import numpy as np
import pandas as pd
import tensorflow as tf
%load_ext tensorboard
from tensorflow.keras import Model
from tensorflow.keras.layers import LSTM, Dropout, Dense, Concatenate, Conv1D, Flatten, Conv2D
import src.data.utils as data_utils
import src.prediction.eval_tools as eval_tools


tf.random.set_seed(17)

print("Available GPUs: ", len(tf.config.list_physical_devices('GPU')))

In [ ]:
def get_windowified_dataset(scale_weights: bool) -> tuple:
    # Generate basic datasets.
    train_dates = pd.date_range("01/01/2016", "31/12/2018")
    test_dates = pd.date_range("01/01/2019", "31/12/2019")
    train_grid = get_dataset(train_dates)
    test_grid = get_dataset(test_dates)

    norm_train_grid, norm_test_grid = normalize_dataset(train_grid, test_grid)

    train_inputs, train_labels = data_utils.generate_data_windows(
        norm_train_grid, train_grid, input_timesteps=7)
    test_inputs, test_labels = data_utils.generate_data_windows(
        norm_test_grid, test_grid, input_timesteps=7)

    # One-Hot Encode Labels.
    train_labels = one_hot_encode_labels(train_labels)
    test_labels = one_hot_encode_labels(test_labels)

    # Get sample weights.
    sample_weights = data_utils.calculate_sample_weights(data=(train_inputs,
                                                               train_labels),
                                                         scale=scale_weights)

    return train_inputs, train_labels, test_inputs, test_labels, sample_weights


def get_dataset(dates: pd.DatetimeIndex) -> tuple:
    data = data_utils.get_dataset(
        date_range=dates,
        auxiliary_data=["weather", "events"],
        encode_event_data=True,
    )
    return data


def one_hot_encode_labels(raw_labels: np.ndarray) -> np.ndarray:
    new_labels = np.empty([len(raw_labels), 2], dtype=np.int8)
    for i, label in enumerate(raw_labels):
        if label == 0:
            new_labels[i] = np.asarray([1, 0], dtype=np.int8)
        else:
            new_labels[i] = np.asarray([0, 1], dtype=np.int8)
    return new_labels


def normalize_dataset(train_grid: pd.DataFrame,
                      test_grid: pd.DataFrame) -> tuple:
    # Normalize breakin values. We normalize on the training data maximum.
    maximum_breakins = data_utils.determine_global_max(train_grid)
    norm_train_grid = data_utils.scale_breakin_values(
        train_grid.copy(deep=True), maximum_breakins)
    norm_test_grid = data_utils.scale_breakin_values(test_grid.copy(deep=True),
                                                     maximum_breakins)

    # Normalize weather data.
    norm_train_grid, norm_test_grid = data_utils.scale_weather_values(
        norm_train_grid, norm_test_grid)

    return norm_train_grid, norm_test_grid

In [ ]:
input_train, labels_train, input_test, labels_test, sample_weights = get_windowified_dataset(
    scale_weights=True)

In [ ]:
import math
import numpy as np
import tensorflow as tf
import src.prediction.eval_tools as eval_tools


def run_through_training_pipeline(
    model: tf.keras.Model,
    input_train: np.ndarray,
    labels_train: np.ndarray,
    input_test: np.ndarray,
    labels_test: np.ndarray,
    sample_weights: np.ndarray = None,
    run_id: int = RUN_ID,
    batch_size: int = BATCH_SIZE,
    epochs: int = EPOCHS,
    initial_lr: float = INITIAL_LR,
    end_lr: float = END_LR,
    decay_steps: int = None,
    shuffle: bool = True,
):
    log_dir = f"logs/lstm/run_{run_id}"

    if not decay_steps:
        decay_steps = math.floor(input_train[0].shape[0] / batch_size) * epochs

    lr_schedule = tf.keras.optimizers.schedules.PolynomialDecay(
        initial_learning_rate=initial_lr,
        end_learning_rate=end_lr,
        decay_steps=decay_steps,
    )

    model.compile(
        loss=tf.keras.losses.BinaryCrossentropy(),
        optimizer=tf.optimizers.Adam(learning_rate=lr_schedule),
        metrics=["accuracy"],
    )

    callbacks = [
        tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
    ]

    model.fit(x=input_train,
              y=labels_train,
              sample_weight=sample_weights,
              shuffle=shuffle,
              batch_size=batch_size,
              validation_data=(input_test, labels_test),
              epochs=epochs,
              callbacks=callbacks)

    predictions_test = model.predict(input_test)
    eval_tools.calculate_metrics(predictions_test, labels_test)


def transform_to_single_step(inputs: list) -> np.ndarray:
    """
    Takes a regular dataset (windowified for multiple input timesteps) and instead turns it into
    single timestep inputs.
    """
    single_timestep_inputs = []
    # Iterate over inputs, which is a list of ndarrays.
    for input_array in inputs:
        # Skip target cell input array because it has no time window dimension.
        if len(input_array.shape) == 2:
            single_timestep_inputs.append(input_array)
            continue
        # Only use last timestep of each data window and reduce dimensionality by 1.
        reduced_input = input_array[:, -1, :].reshape(
            [input_array.shape[0], input_array.shape[-1]])
        single_timestep_inputs.append(reduced_input)
    return single_timestep_inputs


In [ ]:
single_step_input_train = transform_to_single_step(input_train)
single_step_input_test = transform_to_single_step(input_test)

In [ ]:
class SpatialNetwork(tf.keras.Model):
    """
    A simple, single-timestep dense classifier.
    """

    def __init__(self):
        super(SpatialNetwork, self).__init__()
        self.input_breakins = tf.keras.layers.Dense(units=25)
        # self.input_date = tf.keras.layers.Dense(units=2)
        # self.input_weather = tf.keras.layers.Dense(units=11)
        # self.input_events = tf.keras.layers.Dense(units=9)
        self.input_target_cell = tf.keras.layers.Dense(units=25)
        self.concat = tf.keras.layers.Concatenate()
        self.hidden_layer_1 = tf.keras.layers.Dense(units=50,
                                                    activation="ReLU")
        self.dropout_1 = tf.keras.layers.Dropout(rate=0.2)
        self.hidden_layer_2 = tf.keras.layers.Dense(units=50,
                                                    activation="ReLU")
        self.dropout_2 = tf.keras.layers.Dropout(rate=0.2)
        self.hidden_layer_3 = tf.keras.layers.Dense(units=50,
                                                    activation="ReLU")
        self.dropout_3 = tf.keras.layers.Dropout(rate=0.2)
        self.hidden_layer_4 = tf.keras.layers.Dense(units=50,
                                                    activation="ReLU")
        self.dropout_4 = tf.keras.layers.Dropout(rate=0.2)
        self.hidden_layer_5 = tf.keras.layers.Dense(units=50,
                                                    activation="ReLU")
        self.dropout_5 = tf.keras.layers.Dropout(rate=0.2)
        self.hidden_layer_6 = tf.keras.layers.Dense(units=25,
                                                    activation="ReLU")
        self.output_layer = tf.keras.layers.Dense(units=2,
                                                  activation="softmax")

    def call(self, inputs):
        x = self.digest_features(inputs)
        return self.output_layer(x)

    def digest_features(self, inputs):
        input_breakins = self.input_breakins(inputs[0])
        # input_date = self.input_date(inputs[1])
        # input_weather = self.input_weather(inputs[2])
        # input_events = self.input_events(inputs[3])
        input_targets = self.input_target_cell(inputs[4])
        x = self.concat([
            input_breakins,  # input_date, input_weather, input_events,
            input_targets
        ])
        x = self.hidden_layer_1(x)
        x = self.dropout_1(x)
        x = self.hidden_layer_2(x)
        x = self.dropout_2(x)
        x = self.hidden_layer_3(x)
        x = self.dropout_3(x)
        x = self.hidden_layer_4(x)
        x = self.dropout_4(x)
        x = self.hidden_layer_5(x)
        x = self.dropout_5(x)
        return self.hidden_layer_6(x)

In [ ]:
spatial_network = SpatialNetwork()
run_through_training_pipeline(
    model=spatial_network,
    epochs=50,
    input_train=single_step_input_train,
    labels_train=labels_train,
    input_test=single_step_input_test,
    labels_test=labels_test,
    # sample_weights=sample_weights,
    shuffle=False,
    batch_size=25)

In [ ]:
def prepare_input(input) -> list:
    breakins_unprocessed = input[0]
    num_samples = breakins_unprocessed.shape[0]
    timesteps = breakins_unprocessed.shape[1]
    breakins_processed = np.empty((num_samples, timesteps, 25),
                                  dtype=np.float32)
    date = input[1]
    weather = input[2]
    events = input[3]
    target_cell = input[4]
    for i in range(num_samples):
        breakins = breakins_unprocessed[i]
        target_cell_temp = np.array([target_cell[i]] * timesteps)
        breakins_processed[i] = spatial_network.digest_features(
            (breakins, date[i], weather[i], events[i], target_cell_temp))
    return (breakins_processed, date, weather, events, target_cell)

In [ ]:
input_test_new = prepare_input(input_test)

In [ ]:
import pickle

with open("test_data.pkl", "wb") as f:
    pickle.dump((input_test_new, labels_test), f)

In [ ]:
input_train_new = prepare_input(input_train)

In [ ]:
import pickle

with open("train_data.pkl", "wb") as f:
    pickle.dump((input_train_new, labels_train, sample_weights), f)

In [ ]:
def prepare_dataset(input, labels, timesteps_per_input=7):
    new_dataset_length = labels.shape[0] - timesteps_per_input
    # Decompose input tuple into components.
    breakins, dates, weather, events, target_cell = input
    breakins_new = np.empty(
        (new_dataset_length, timesteps_per_input, breakins.shape[-1]))
    dates_new = np.empty(
        (new_dataset_length, timesteps_per_input, dates.shape[-1]))
    weather_new = np.empty(
        (new_dataset_length, timesteps_per_input, weather.shape[-1]))
    events_new = np.empty(
        (new_dataset_length, timesteps_per_input, events.shape[-1]))
    target_cell_new = np.empty(
        (new_dataset_length, timesteps_per_input, target_cell.shape[-1]))
    labels_new = np.empty((new_dataset_length, labels.shape[1]))
    for sample_index in range(new_dataset_length):
        for timestep_index in range(timesteps_per_input):
            breakins_new[sample_index,
                         timestep_index] = breakins[sample_index,
                                                    timestep_index]
            dates_new[sample_index,
                      timestep_index] = dates[sample_index + timestep_index]
            weather_new[sample_index, timestep_index] = weather[sample_index +
                                                                timestep_index]
            events_new[sample_index,
                       timestep_index] = events[sample_index + timestep_index]
            target_cell_new[sample_index,
                            timestep_index] = target_cell[sample_index +
                                                          timestep_index]
        labels_new[sample_index] = labels[sample_index + timesteps_per_input]
    input_new = (breakins_new, dates_new, weather_new, events_new,
                 target_cell_new)
    digested_input = spatial_network.digest_features(input_new).numpy()
    return digested_input, labels_new

In [ ]:
import pickle
with open("train_data.pkl", "rb") as f:
    input_train_new, labels_train, sample_weights = pickle.load(f)
with open("test_data.pkl", "rb") as f:
    input_test_new, labels_test = pickle.load(f)

In [ ]:
class TemporalNetwork(tf.keras.Model):
    """
    A classifier feeding deep stuff to an LSTM.
    """

    def __init__(self):
        super(TemporalNetwork, self).__init__()
        self.lstm = tf.keras.layers.LSTM(25, return_sequences=True)
        self.flatten = tf.keras.layers.Flatten()
        self.input_date = tf.keras.layers.Dense(units=2)
        self.input_weather = Dense(units=11)
        self.input_events = Dense(units=9)
        self.input_target_cell = tf.keras.layers.Dense(units=25)
        self.concatenate = tf.keras.layers.Concatenate()
        self.hidden_layer_1 = tf.keras.layers.Dense(units=100,
                                                    activation="ReLU")
        self.hidden_layer_2 = tf.keras.layers.Dense(units=50,
                                                    activation="ReLU")
        self.hidden_layer_3 = tf.keras.layers.Dense(units=25,
                                                    activation="ReLU")
        self.output_layer = tf.keras.layers.Dense(units=2,
                                                  activation="softmax")

    def call(self, inputs):
        x = self.lstm(inputs[0])
        x = self.flatten(x)
        date = self.input_date(inputs[1])
        weather = self.input_weather(inputs[2])
        events = self.input_events(inputs[3])
        target_cell = self.input_target_cell(inputs[4])
        x = self.concatenate([x, date, weather, events, target_cell])
        x = self.hidden_layer_1(x)
        x = self.hidden_layer_2(x)
        x = self.hidden_layer_3(x)
        return self.output_layer(x)


temporal_network = TemporalNetwork()

In [ ]:
run_through_training_pipeline(
    model=temporal_network,
    epochs=5,
    input_train=input_train_new,
    labels_train=labels_train,
    input_test=input_test_new,
    labels_test=labels_test,
    # sample_weights=sample_weights,
    decay_steps=100,
)


In [ ]:
class TemporalNetwork(tf.keras.Model):
    """
    A classifier feeding deep stuff to an LSTM.
    """

    def __init__(self):
        super(TemporalNetwork, self).__init__()
        self.input_day_0 = Dense(units=25)
        self.input_day_1 = Dense(units=25)
        self.input_day_2 = Dense(units=25)
        self.input_day_3 = Dense(units=25)
        self.input_day_4 = Dense(units=25)
        self.input_day_5 = Dense(units=25)
        self.input_day_6 = Dense(units=25)
        self.concatenate = tf.keras.layers.Concatenate()
        self.input_date = tf.keras.layers.Dense(units=2)
        self.input_weather = Dense(units=11)
        self.input_events = Dense(units=9)
        self.input_target_cell = tf.keras.layers.Dense(units=25)
        self.hidden_layer_1 = tf.keras.layers.Dense(units=100,
                                                    activation="ReLU")
        self.hidden_layer_2 = tf.keras.layers.Dense(units=50,
                                                    activation="ReLU")
        self.hidden_layer_3 = tf.keras.layers.Dense(units=25,
                                                    activation="ReLU")
        self.output_layer = tf.keras.layers.Dense(units=2,
                                                  activation="softmax")

    def call(self, inputs):
        x0 = self.input_day_0(inputs[0][:, 0])
        x1 = self.input_day_1(inputs[0][:, 1])
        x2 = self.input_day_2(inputs[0][:, 2])
        x3 = self.input_day_3(inputs[0][:, 3])
        x4 = self.input_day_4(inputs[0][:, 4])
        x5 = self.input_day_5(inputs[0][:, 5])
        x6 = self.input_day_6(inputs[0][:, 6])
        x = self.concatenate([x0, x1, x2, x3, x4, x5, x6])
        date = self.input_date(inputs[1])
        weather = self.input_weather(inputs[2])
        events = self.input_events(inputs[3])
        target_cell = self.input_target_cell(inputs[4])
        x = self.concatenate([x, date, weather, events, target_cell])
        x = self.hidden_layer_1(x)
        x = self.hidden_layer_2(x)
        x = self.hidden_layer_3(x)
        return self.output_layer(x)


temporal_network = TemporalNetwork()

In [ ]:
run_through_training_pipeline(
    model=temporal_network,
    epochs=5,
    input_train=input_train_new,
    labels_train=labels_train,
    input_test=input_test_new,
    labels_test=labels_test,
    # sample_weights=sample_weights,
    decay_steps=100,
)


In [ ]:
class TemporalNetwork(tf.keras.Model):
    """
    A classifier feeding deep stuff to an LSTM.
    """

    def __init__(self):
        super(TemporalNetwork, self).__init__()
        self.convolutional_layer = tf.keras.layers.Conv1D(strides=25,
                                                          filters=50,
                                                          activation="relu",
                                                          kernel_size=7)
        self.flatten = tf.keras.layers.Flatten()
        self.input_date = tf.keras.layers.Dense(units=2)
        self.input_weather = Dense(units=11)
        self.input_events = Dense(units=9)
        self.input_target_cell = tf.keras.layers.Dense(units=25)
        self.concatenate = tf.keras.layers.Concatenate()
        self.hidden_layer_1 = tf.keras.layers.Dense(units=100,
                                                    activation="ReLU")
        self.hidden_layer_2 = tf.keras.layers.Dense(units=50,
                                                    activation="ReLU")
        self.hidden_layer_3 = tf.keras.layers.Dense(units=25,
                                                    activation="ReLU")
        self.output_layer = tf.keras.layers.Dense(units=2,
                                                  activation="softmax")

    def call(self, inputs):
        x = self.convolutional_layer(inputs[0])
        x = self.flatten(x)
        date = self.input_date(inputs[1])
        weather = self.input_weather(inputs[2])
        events = self.input_events(inputs[3])
        target_cell = self.input_target_cell(inputs[4])
        x = self.concatenate([x, date, weather, events, target_cell])
        x = self.hidden_layer_1(x)
        x = self.hidden_layer_2(x)
        x = self.hidden_layer_3(x)
        return self.output_layer(x)


temporal_network = TemporalNetwork()

In [ ]:
run_through_training_pipeline(
    model=temporal_network,
    epochs=5,
    input_train=input_train_new,
    labels_train=labels_train,
    input_test=input_test_new,
    labels_test=labels_test,
    # sample_weights=sample_weights,
    decay_steps=100,
)
